In [31]:
using LinearAlgebra

In [32]:
function fatoracao_aprox(A, k, passos)
    """Recebe uma matriz A(m,n), um k natural > 0 e um número de passos.
       Retorna matrizes B(m,k) e C(k,n) tal que A é aproximadamente B*C e B*C tem posto k."""
    
    local C
    m, n = size(A)
    B = randn(m,k)  # sorteio da base inicial
    
    # Mínimos Quadrados alternados
    for i=1:passos 
        C = B\A
        B = A/C    
    end
    erro = norm(A - B*C)
    
    return B, C, erro
end

fatoracao_aprox (generic function with 1 method)

fatoracao_aprox (generic function with 1 method)

In [33]:
# (TESTE) FATORAÇÃO APROXIMADA

m, n = 4, 6
k, passos = 1, 1000
A = randn(m, n)
B, C, erro = fatoracao_aprox(A, k, passos)

println("Número de passos = ", passos)
println("posto(A) = ", rank(A))
println("posto(BC) = ", rank(B*C))
println("Erro = ", erro)

Número de passos = 1000
posto(A) = 4
posto(BC) = 1
Erro = 3.2464566074925285
Número de passos = 1000
posto(A) = 4
posto(BC) = 1
Erro = 3.2464566074925285


In [34]:
function LU(A) 
    """Recebe uma matriz quadrada A e a decompõe em duas matrizes: L e U, onde 
       L é uma matriz triangular inferior e U uma matriz triangular superior."""
    
    n, n = size(A)
    L = zeros(n, n)
    U = zeros(n, n)
    
    for i=1:n
        coluna = (1 / A[i,i]) * A[:,i]  # copia coluna i de A e divide pelo pivô: A[i,i]
        L[:,i] = coluna   # adiciona em L
        
        linha = A[i,:]  # copia linha i de A
        U[i,:] = linha  # adiciona em U
        
        A = A - (coluna*linha')  # atualiza matriz A
    end
    
    return L,U
end

LU (generic function with 1 method)

LU (generic function with 1 method)

In [35]:
# (TESTE) DECOMPOSIÇÃO LU

n = 3
A = randn(n, n)
L, U = LU(A)
erro = norm(A - L*U)

1.734723475976807e-18

1.734723475976807e-18

In [36]:
function subs_reversa(U, b) 
    """Recebe uma matriz U quadrada triangular superior e um vetor b. 
       Retorna o vetor x da equação Ux = b"""
    
    n, n = size(U)
    x = zeros(n, 1)
    
    for i=n:-1:1    # vai de n até 1
        x[i] = (1/U[i,i])*(b[i] - norm(U[i,:]'x))  # no laço i, cada elemento de x[i] até x[1] ainda será 0.
    end
    
    return x 
end

subs_reversa (generic function with 1 method)

subs_reversa (generic function with 1 method)

In [37]:
# (TESTE) SUBSTITUIÇÃO REVERSA

n = 3
A = randn(n, n)
L,U = LU(A)
b = randn(n, 1)
x = subs_reversa(U, b)
erro = norm(U*x - b)

5.551115123125783e-17

5.551115123125783e-17

In [38]:
function subs_direta(L, b)
    """Recebe uma matriz L quadrada triangular inferior e um vetor b. 
       Retorna o vetor x da equação Lx = b"""
    
    n, n = size(L)
    x = zeros(n, 1)
    
    for i=1:n
        x[i] = (1/L[i,i])*(b[i] - norm(L[i,:]'x))  # no laço i, cada elemento de x[i] até x[n] ainda será 0.
    end
    
    return x 
end

subs_direta (generic function with 1 method)

subs_direta (generic function with 1 method)

In [39]:
# (TESTE) SUBSTITUIÇÃO DIRETA

n = 3
A = randn(n, n)
L,U = LU(A)
b = randn(n, 1)
x = subs_direta(L, b)
erro = norm(L*x - b)

3.3306690738754696e-16

3.3306690738754696e-16

In [40]:
function resolve_sistema(A, b)
    """Recebe uma matriz quadrada A (NxN) e um vetor b (Nx1). 
       Utilizando a decomposição LU e as funções de substituição, retorna o vetor x da equação Ax = b"""
    
    L,U = LU(A)
    y = subs_direta(L, b)
    x = subs_reversa(U, y)
    return x
end

resolve_sistema (generic function with 1 method)

resolve_sistema (generic function with 1 method)

In [41]:
# (TESTE) RESOLVE SISTEMA

n = 3
A = randn(n, n)
b = randn(n, 1)
x = resolve_sistema(A, b)
erro = norm(A*x - b)

23.77242683835294

23.77242683835294

In [42]:
function normalizar(v)
    """Recebe um vetor v e o retorna normalizado (norma = 1)."""
    
    return (v / norm(v))
end

normalizar (generic function with 1 method)

normalizar (generic function with 1 method)

In [43]:
# (TESTE) NORMALIZAR

n = 3
v = randn(n)
norm(normalizar(v)) # deve ser igual a 1

1.0

1.0

In [44]:
function QR(A)
    """Recebe uma matriz A e a decompõe em duas matrizes: Q e R, onde 
       Q é uma matriz ortogonal e R uma matriz triangular superior."""
    
    m, n = size(A)  # aceita matrizes não quadradas
    Q = zeros(m, n)  
    R = zeros(n, n)
     
    for i=1:n
        coluna = normalizar(A[:,i])   # obtém coluna i de Q (normalizada)
        Q[:,i] = coluna
        linha = A'*coluna   # obtém linha i de R
        R[i,:] = linha
        A = A - coluna*linha'  # zera coluna i de A
    end
    
    return Q,R
end

QR (generic function with 1 method)

QR (generic function with 1 method)

In [45]:
# (TESTE) DECOMPOSIÇÃO QR

m, n = 4, 3
A = randn(m, n)
Q,R = QR(A)
erro = norm(A - Q*R)

8.182012274917481e-16

8.182012274917481e-16

In [46]:
function projetar(a, Q)
    """Recebe um vetor 'a' e projeta este na direção de cada coluna (normalizada) da matriz Q. 
       Retorna o vetor 'p' que é a soma de todas essas projeções."""
    
    if (Q isa Vector{<:Number})  # caso Q seja um vetor, será intepretado como uma matriz
        Q = reshape(Q, length(Q), 1)
    end
    
    m, n = size(Q)
    p = zeros(m)
    
    for i=1:n
        direcao = normalizar(Q[:,i])
        p += (a'*direcao)*direcao
    end
    
    return p
end

projetar (generic function with 1 method)

projetar (generic function with 1 method)

In [47]:
# (TESTE) PROJETAR

m, n = 3, 4
a = randn(m)
Q = randn(m, n)
projetar(a, Q)

3-element Vector{Float64}:
 -0.5828524570472329
 -4.297326617637275
 -1.2884680843325356

3-element Vector{Float64}:
 -0.5828524570472329
 -4.297326617637275
 -1.2884680843325356

In [48]:
function QR_classico(A)
    """Recebe uma matriz A e a decompõe em duas matrizes: Q e R, onde 
       Q é uma matriz ortogonal e R uma matriz triangular superior."""
    
    m, n = size(A)  # aceita matrizes não quadradas
    Q = zeros(m, n)  
    R = zeros(n, n)
    
    A1 = A[:,1]
    Q[:,1], R[1,1] = normalizar(A1), norm(A1)
    for i=2:n
        Ai = A[:,i] 
        Q[:,i] = normalizar(Ai - projetar(Ai, Q[:, 1:(i-1)]))  # obtém coluna i de Q (normalizada)
        
        # constrói matriz R
        for j=1:i    
            if !(i < j) 
                R[j,i] = Ai'*Q[:,j]
            end
        end
    end
    
    return Q,R
end

QR_classico (generic function with 1 method)

QR_classico (generic function with 1 method)

In [49]:
# (TESTE) QR CLÁSSICO

m, n = 4, 3
A = randn(m, n)
Q, R = QR_classico(A)
erro = norm(A - Q*R)

4.972820174300706e-16

4.972820174300706e-16

In [50]:
function MQ(A, b)
    """Recebe uma matriz A(m,n) e um vetor b(m,1). Utiliza decomposição QR e 
       substituição reversa para encontrar o vetor x tal que A*x é aproximadamente b."""
    
    C = [A b]  # concatena A e b
    Q,R = QR(C)  # decompõe C em Q*R
    n, n = size(R) # R é uma matriz quadrada
    R = R[1:(n-1), :] # elimina a última linha de R
    y = R[:, n]  # guarda a última coluna de R em y
    C = R[1:(n-1), 1:(n-1)] # guarda o restante de R em C
    x = subs_reversa(C, y) # C agora é uma matriz quadrada triangular superior: (U)
    
    erro = norm(A*x - b) # encontra erro da aproximação
    return x, erro
end

MQ (generic function with 1 method)

MQ (generic function with 1 method)

In [51]:
# (TESTE) MÍNIMOS QUADRADOS COM QR

m, n = 5, 4  # mais linhas que colunas -> sistema sem solução
A = randn(m, n)
b = randn(m)
x, erro = MQ(A, b)
erro

0.8820458169507486

0.8820458169507486

In [1]:
# ACHAR INVERSA DE A

A = [2 1 1; 4 5 4; 6 15 16]
e1 = [1;0;0]
e2 = [0;1;0]
e3 = [0;0;1]
x1 = resolve_sistema(A, e1)
x2 = resolve_sistema(A, e2)
x3 = resolve_sistema(A, e3)
x = [x1 x2 x3]

LoadError: UndefVarError: resolve_sistema not defined

In [55]:
function eh_triang_superior(A)
   n, n = size(A)
    
    for i=1:n
        for j=1:i
            if (i == j) && (A[i,j] != 1)  return false
            end
            if (i > j) && (A[i,j] != 0)  return false
            end
        end
    end
        
    return true
end

eh_triang_superior (generic function with 1 method)

eh_triang_superior (generic function with 1 method)

In [56]:
function eh_triang_sup(A) # recursivo
   n, n = size(A)
    
    if (n == 1)
        if (A[n, n] == 1) return true
        end
    else
        if(eh_triang_sup(A[1:n-1, 1:n-1]))
                if (A[n, 1:n-1]' == zeros(1, n-1)) && A[n,n] == 1
                    return true
                end
        end
    end
    
    return false
end

eh_triang_sup (generic function with 1 method)

eh_triang_sup (generic function with 1 method)

In [57]:
id = Matrix{Float64}(I, 4, 4)
eh_triang_superior(id) && eh_triang_sup(id)

true

true

In [58]:
function Rev(U, b, x, i)
    n, n = size(U)
    
    if (i == n) 
        x[n] = b[n] / U[n, n]
        return x[n], x
    else
        v = zeros(n-i)
        for j=1:(n-i)
            v[j], x = Rev(U, b, x, i+j)
        end
        x[i] = (b[i] - norm(U[i,(i+1):n]'v)) / U[i, i]
        return x[i], x
    end
end

Rev (generic function with 1 method)

Rev (generic function with 1 method)

In [59]:
U = [1 2 3; 0 1 4; 0 0 1]
b = [3; 4; 5]
subs_reversa(U, b)

3×1 Matrix{Float64}:
 -14.0
 -16.0
   5.0

3×1 Matrix{Float64}:
 -14.0
 -16.0
   5.0

In [60]:
x = zeros(3)
x1, x = Rev(U, b, x, 1)
x

3-element Vector{Float64}:
 -14.0
 -16.0
   5.0

3-element Vector{Float64}:
 -14.0
 -16.0
   5.0